In [1]:
## IMPORTS

import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, precision_score, f1_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

# import custom modules
import sys
sys.path.insert(1, '../utils/')
import model_utils

In [2]:
df = pd.read_csv("../../pre_processing/datasets/no_filtered_analysis.csv")

In [3]:
# Drop da coluna de RA
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

Removing column RA


In [ ]:
df.head()

In [4]:
# Run all predictions based on fold n_splits
def run_prediction(X_train, X_test, y_train, y_test):

    # used to evaluate best model params
    # gridsearch_metrics = {
    #         'REC':'recall', 
    #         'AUC':'roc_auc', 
    #         'Accuracy': 'accuracy'
    #}

    # model gridsearch params
    model_params = {
        'kernel': ['linear','rbf', 'sigmoid'],
        'C': [0.3,0.5,1,2,3,10],
        'gamma': [0.3,0.5,1],
        'decision_function_shape':['ovo', 'ovr'],
        'shrinking': [True, False]
    }

    # gridsearch initialization
    clf = GridSearchCV(SVC(),
                       param_grid = model_params,
                       #scoring = gridsearch_metrics,
                       refit='accuracy')

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    predict_data = {
        "confusion_matrix": confusion_matrix(y_test, predictions),
        "classification_report": classification_report(y_test, predictions),
        "model_accuracy": accuracy_score(y_test, predictions),
        "f1_score": f1_score(y_test, predictions),
        "precision_score": precision_score(y_test, predictions),
        "roc_score": roc_auc_score(y_test, predictions),
        "recall_score": recall_score(y_test, predictions),
        "epoch_params": clf.best_params_
    }

    return predict_data

In [5]:
def main():
    X = df.iloc[:, df.columns != "EVADIDO"].values
    y = df["EVADIDO"]

    skf = StratifiedKFold(n_splits = 4)
    skf.get_n_splits(X, y)
    scaler = StandardScaler()
    print(f"StratifiedKFold config: {skf} \n")
    all_predictions = []
    epoch = 1
    for train_index, test_index in skf.split(X, y):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # return best k element
        all_predictions.append(run_prediction(X_train, X_test, y_train, y_test))
        
    # General model outputs
    model_utils.generate_output(all_predictions)
    # CSV Output results
    model_utils.generate_output_csv(all_predictions)
    
    print("############################ FINISHED ############################")

In [6]:
if __name__ == "__main__":
    main()

StratifiedKFold config: StratifiedKFold(n_splits=4, random_state=None, shuffle=False) 



/home/dlpires/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/dlpires/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/home/dlpires/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/dlpires/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The d